# 云打码官方接口

下面代码是云打码官方提供的接口，其中需要自己配置的地方是：
- username = 'whoarewe' # 用户名

- password = '***' # 密码

- appid = 4283 # appid

- appkey = '02074c64f0d0bb9efb2df455537b01c3' # appkey

- codetype = 1004 # 验证码类型

In [ ]:
import requests
import json
import time


class YDMHttp:
    apiurl = 'http://api.yundama.com/api.php'
    username = ''
    password = ''
    appid = ''
    appkey = ''

    def __init__(self, username, password, appid, appkey):
        self.username = username
        self.password = password
        self.appid = str(appid)
        self.appkey = appkey

    def request(self, fields, files=[]):
        response = self.post_url(self.apiurl, fields, files)
        response = json.loads(response)
        return response

    def balance(self):
        data = {'method': 'balance', 'username': self.username, 'password': self.password, 'appid': self.appid,
                'appkey': self.appkey}
        response = self.request(data)
        if (response):
            if (response['ret'] and response['ret'] < 0):
                return response['ret']
            else:
                return response['balance']
        else:
            return -9001

    def login(self):
        data = {'method': 'login', 'username': self.username, 'password': self.password, 'appid': self.appid,
                'appkey': self.appkey}
        response = self.request(data)
        if (response):
            if (response['ret'] and response['ret'] < 0):
                return response['ret']
            else:
                return response['uid']
        else:
            return -9001

    def upload(self, filename, codetype, timeout):
        data = {'method': 'upload', 'username': self.username, 'password': self.password, 'appid': self.appid,
                'appkey': self.appkey, 'codetype': str(codetype), 'timeout': str(timeout)}
        file = {'file': filename}
        response = self.request(data, file)
        if (response):
            if (response['ret'] and response['ret'] < 0):
                return response['ret']
            else:
                return response['cid']
        else:
            return -9001

    def result(self, cid):
        data = {'method': 'result', 'username': self.username, 'password': self.password, 'appid': self.appid,
                'appkey': self.appkey, 'cid': str(cid)}
        response = self.request(data)
        return response and response['text'] or ''

    def decode(self, filename, codetype, timeout):
        cid = self.upload(filename, codetype, timeout)
        if (cid > 0):
            for i in range(0, timeout):
                result = self.result(cid)
                if (result != ''):
                    return cid, result
                else:
                    time.sleep(1)
            return -3003, ''
        else:
            return cid, ''

    def post_url(self, url, fields, files=[]):
        # for key in files:
        #     files[key] = open(files[key], 'rb');
        res = requests.post(url, files=files, data=fields)
        return res.text


username = 'whoarewe'  # 用户名

password = '***'  # 密码

appid = 4283  # appid

appkey = '02074c64f0d0bb9efb2df455537b01c3'  # appkey

filename = 'getimage.jpg'  # 文件位置

codetype = 1004  # 验证码类型

# 超时
timeout = 60


def indetify(response_content):
    if (username == 'username'):
        print('请设置好相关参数再测试')
    else:
        # 初始化
        yundama = YDMHttp(username, password, appid, appkey)

        # 登陆云打码
        uid = yundama.login()
        print('uid: %s' % uid)

        # 查询余额
        balance = yundama.balance()
        print('balance: %s' % balance)

        # 开始识别，图片路径，验证码类型ID，超时时间（秒），识别结果
        cid, result = yundama.decode(response_content, codetype, timeout)
        print('cid: %s, result: %s' % (cid, result))
        return result


def indetify_by_filepath(file_path):
    if (username == 'username'):
        print('请设置好相关参数再测试')
    else:
        # 初始化
        yundama = YDMHttp(username, password, appid, appkey)

        # 登陆云打码
        uid = yundama.login()
        print('uid: %s' % uid)

        # 查询余额
        balance = yundama.balance()
        print('balance: %s' % balance)

        # 开始识别，图片路径，验证码类型ID，超时时间（秒），识别结果
        cid, result = yundama.decode(file_path, codetype, timeout)
        print('cid: %s, result: %s' % (cid, result))
        return result


if __name__ == '__main__':
    pass


下面以豆瓣登录过程中的验证码为例，了解云打码如何使用。

In [ ]:
from selenium import webdriver
import time
import requests
from yundama import indetify

driver = webdriver.Chrome()

driver.get("https://www.douban.com/")

# 输入用户名
driver.find_element_by_id("form_email").send_keys("78****@qq.com")

# 输入密码
driver.find_element_by_id("form_password").send_keys("****")

# 获取验证码的地址
img_url = driver.find_element_by_id("captcha_image").get_attribute("src")
response = requests.get(img_url)  # 请求验证码的地址
ret = indetify(response.content)  # 验证码识别

# 输入验证码
driver.find_element_by_id("captcha_field").send_keys(ret)


time.sleep(5)
# 点击登录
driver.find_element_by_class_name("bn-submit").click()


time.sleep(10)
print(driver.get_cookies())
driver.quit()
